In [ ]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):
    import re

    infix_re = compile_prefix_regex(nlp.Defaults.infixes)
    prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

nlp = spacy.load('en_core_web_lg') 
nlp.tokenizer = custom_tokenizer(nlp)


In [ ]:
import gzip, pickle

from utils.grammar import generate_candidates, iterate_all_patterns, remove_overlap

In [ ]:
class DotDict(dict):
    
    def __init__(self, token, doc):
        index, text, lemma, norm, pos, tag, dep, head, children = token.split('|')
        children = children.strip()

        self.i = int(index)
        self.text = text
        self.norm_ = norm
        self.lemma_ = lemma
        self.dep_ = dep
        self.pos_ = pos
        self.tag_ = tag
        self.head_ = int(head)
        self.children_ = children
        self.doc = doc
    
    def __getattr__(self, name):
        if name == 'head':
            return self.doc[head_]
        elif name == 'children':
            return [self.doc[int(e)] for e in self.children_.split(',')] if self.children_ else []
        else:
            return self[name]
    
    
class Parse(list):

    def __init__(self, entry):
        tokens = [token for token in entry.strip().split('\t') if token]
        text, tokens = tokens[0], tokens[1:]
        
        self.extend([DotDict(token, self) for token in tokens])

In [ ]:
fs = gzip.open('bnc.parse.txt.gz', 'rt', encoding='utf8')

In [ ]:
from itertools import product
import json

def duplicate_sent(sent):
    sent = sent.replace('\\', '')
    tokens = []
    for token in sent.split():
        tokens.append(token.split('/') if '/' in token else [token])

    composes = product(*tokens)
    sents = [' '.join(compose) for compose in composes]
    
    return sents

def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try: results.append(a_dict[id])
        except KeyError: pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict)  # Return value ignored.
    results = [s for result in results for sent in result for s in duplicate_sent(sent) ] # flatten
    
    return results


In [ ]:
# camb_dict = json.dumps(json.load(open('data/cambridge.dict.json')))
# camb_ph_dict = json.dumps(json.load(open('data/cambridge.dict.phrase.json')))
# dict_sentences = find_values('dic_examples', camb_dict) + find_values('dic_examples', camb_ph_dict)


In [ ]:
from collections import defaultdict, Counter

ngrams = defaultdict(Counter)
counts = Counter()
sentences = defaultdict(list)

# for i, entry in enumerate(fs):
for i, sentence in enumerate(dict_sentences):
    # parse = Parse(entry)
    parse = nlp(sentence, disable=['ner'])

    # 1. generate possible sentences
    parses = generate_candidates(parse)

    # 2. find patterns for each candidate
    gets = [get for parse in parses for get in iterate_all_patterns(parse)]

    # 3. remove duplicate
    gets = remove_overlap(parse, gets)

    text = ' '.join([tk.text for tk in parse])
    for get in gets:
        counts[(get['match'], get['no'])] += 1
        ngrams[(get['match'], get['no'])][get['ngram']] += 1
        sentences[get['ngram']].append(text)
    
    if i % 10000 == 0:
        print(i)
        with open('new.recommend.pickle', 'wb') as handle:
            pickle.dump({ 'counts': counts, 'ngrams': ngrams, 'sentences': sentences }, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('new.recommend.pickle', 'wb') as handle:
    pickle.dump({ 'counts': counts, 'ngrams': ngrams, 'sentences': sentences }, handle, protocol=pickle.HIGHEST_PROTOCOL)